In [ ]:
import sys
import torch
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

sns.set_theme(style="white")

# GDSC & Gene Expression Insights

This notebook is intended to get an overview about the data sources provided by the GDSC database.

In [ ]:
device = torch.device("cpu")

print(f"""
    Python version:   {sys.version}
    PyTorch version:  {torch.__version__}
    Device:           {device}
    CUDA available:   {torch.cuda.is_available()}
""")

In [ ]:
!pwd
!ls -lh ../../datasets/gdsc/screening_data | grep '/\|[xlsx|csv]$' | sort

Different __Screening Data__ from the [GDSC's download page](https://www.cancerrxgene.org/downloads/bulk_download) has been saved in the folder above. The description to each file compared to the website is as follows: 
- __Drug Screening - IC50s__
  - __GDSC1-dataset__: `GDSC1_fitted_dose_response_25Feb20.xlsx`
  - __GDSC2-dataset__: `GDSC2_fitted_dose_response_25Feb20.xlsx`
- __Drug Screening - Raw data__: 
  - __GDSC1-raw-data__: `GDSC1_public_raw_data_25Feb20.csv`
  - __GDSC2-raw-data__: `GDSC2_public_raw_data_25Feb20.csv`
- __All cell lines screened__: `Cell_Lines_Details.xlsx`

In the following we will investigate each of these data sources.

## Drug Screening

In [ ]:
PATH_TO_GDSC_SCREENING_DATA = '../../datasets/gdsc/screening_data/'
PATH_TO_SAVE_DATA_TO = '../../datasets/gdsc/my_datasets/'

### IC50s

In [ ]:
GDSC1_IC50_FILE = 'GDSC1_fitted_dose_response_25Feb20.xlsx'
GDSC2_IC50_FILE = 'GDSC2_fitted_dose_response_25Feb20.xlsx'

In [ ]:
# Read the IC50 files.

# GDSC1
start = time.time()
gdsc1_ic50s = pd.read_excel(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC1_IC50_FILE}', header=0)
print(f"File `{GDSC1_IC50_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc1_ic50s.shape}")

# GDSC2
start = time.time()
gdsc2_ic50s = pd.read_excel(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC2_IC50_FILE}', header=0)
print(f"File `{GDSC2_IC50_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc2_ic50s.shape}")

In [ ]:
gdsc1_ic50s.head(5)

In [ ]:
gdsc2_ic50s.head(5)

- GDSC1 contains $310,904$ rows
- GDSC2 contains $135,242$ rows

Each dataset contains the same $19$ columns. They hold the following informations (which were taken from [here](ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC_Fitted_Data_Description.pdf)):

- A description of the columns can be found in `knowledge_base/table_contents.md` under `Drug Screening - IC50s`.

The tables contain the IC50 values (`LN_IC50`) for specific drugs (`DRUG_NAME`) on cancer cell lines (`CELL_LINE_NAME`). Thus, they hold information about how specific drugs influence the killing of cancer cells.

#### EDA

__Questions to answer__: 

- [x] How many unique drugs, cell-lines and cell-line-drug combinations are there (per database)? 
- [x] How many number of observations does each drug (`DRUG_NAME`) have (per database)? 
- [x] How many number of observations does each cell line (`CELL_LINE_NAME`) have (per database)?
- [x] How are the IC50 values distributed (per database)?

__Tasks__:
- [x] Thin out genes by using landmark gene list.
- [ ] Build GDSC base table including gene expression data
- [ ] Add to GDSC base table Copy Number Variation data
- [ ] Add to GDSC base table Mutation data

In [ ]:
gdsc2_ic50s[['DRUG_ID', 'CELL_LINE_NAME']].shape

In [ ]:
columns = ['DRUG_NAME', 'CELL_LINE_NAME', ['DRUG_NAME', 'CELL_LINE_NAME'], 'DRUG_ID']

for col in columns:
  gdsc1_uniq, gdsc2_uniq = np.unique(gdsc1_ic50s[col]), np.unique(gdsc2_ic50s[col])

  print(f"""
      Number of unique {col}'s 
          - for GDSC1 is {gdsc1_uniq.size}
          - for GDSC2 is {gdsc2_uniq.size}
          - in total for both is {set(gdsc1_uniq.tolist() + gdsc2_uniq.tolist()).__len__()}
            Thus, {(gdsc1_uniq.size+gdsc2_uniq.size) - set(gdsc1_uniq.tolist() + gdsc2_uniq.tolist()).__len__()} {col}'s are contained in both databases (since the full join is {(gdsc1_uniq.size+gdsc2_uniq.size)}).
  """)

In [ ]:
# Join both datasets for analysis purposes.
gdsc_ic50s_join = pd.concat([gdsc1_ic50s, gdsc2_ic50s], ignore_index=True)
assert gdsc_ic50s_join[gdsc_ic50s_join.index.duplicated()].shape[0] == 0
assert gdsc_ic50s_join.shape[0] == gdsc1_ic50s.shape[0] + gdsc2_ic50s.shape[0]

- GDSC1-raw contains $5,837,703$ rows
- GDSC2-raw contains $6,646,430$ rows

Each dataset contains the same $18$ columns. They hold the following informations:

- A description of the columns can be found in `knowledge_base/table_contents.md` under `Drug Screening - Raw data`.

1. We start by investigating the `DRUG_NAME` column.

In [ ]:
col = 'DRUG_NAME'

In [ ]:
# How many number of observations does each drug have per database? 
gdsc_value_counts = gdsc_ic50s_join[['DATASET', col]].value_counts().to_frame().reset_index()
gdsc_value_counts.rename(columns={0:'counts'}, inplace=True)
gdsc_value_counts.head(-5)

In [ ]:
# Distribution of counts per DRUG_NAME.

figure, axs = plt.subplots(1, 2, figsize=(15, 5))
figure.suptitle(f"Counts of Data Points per {col} for GDSC1 & GDSC2")

sns.boxplot(data=gdsc_value_counts, x='counts', y='DATASET', linewidth=2, ax=axs[0]);
axs[0].set_xlabel(f"Count of Data Points per {col} Group");
sns.kdeplot(data=gdsc_value_counts, x='counts', hue='DATASET', linewidth=2, ax=axs[1]);
axs[1].set_xlabel(f"Count of Data Points per {col} Group");

- The counts per cell-line (`DRUG_NAME`) are not different strongly between the two databases.
- Outliers are going in both directions, smaller as well as larger counts. This means, that there are some drugs for which their number of observations is relatively low and some where its relatively high.

In [ ]:
describes = []
for db in ['GDSC1', 'GDSC2']:
    print(f"\nDatabase `{db}`\n{15*'-'}")
    describe = gdsc_value_counts[gdsc_value_counts.DATASET==db].counts.describe()
    describes.append(describe)
    print(describe)

In [ ]:
print(f"The drugs (`{col}`) for")
for i, db in enumerate(['GDSC1', 'GDSC2']):
    print(f""" 
        - {db} have
            - mostly (IQR) between {round(describes[i]['25%'])} and {round(describes[i]['75%'])} observations
            - on average {round(describes[i]['mean'])} observations per drug. This corresponds to {100*round(describes[i]['mean'])/gdsc_ic50s_join[gdsc_ic50s_join.DATASET==db].shape[0]:2.2f}% out of all observations ({gdsc_ic50s_join[gdsc_ic50s_join.DATASET==db].shape[0]}) in {db}.
    """)

2. Now we investigate analogous the `CELL_LINE_NAME` column.

In [ ]:
col = 'CELL_LINE_NAME'

In [ ]:
# How many number of observations does each drug have per database? 
gdsc_cell_line_value_counts = gdsc_ic50s_join[['DATASET', col]].value_counts().to_frame().reset_index()
gdsc_cell_line_value_counts.rename(columns={0:'counts'}, inplace=True)
gdsc_cell_line_value_counts.head(-5)

In [ ]:
# Distribution of counts per CELL_LINE_NAME.

figure, axs = plt.subplots(1, 2, figsize=(15, 5))
figure.suptitle(f"Counts of Data Points per {col} for GDSC1 & GDSC2")

sns.boxplot(data=gdsc_cell_line_value_counts, x='counts', y='DATASET', linewidth=2, ax=axs[0]);
axs[0].set_xlabel(f"Count of Data Points per {col} Group");
sns.kdeplot(data=gdsc_cell_line_value_counts, x='counts', hue='DATASET', linewidth=2, ax=axs[1]);
axs[1].set_xlabel(f"Count of Data Points per {col} Group");

- The counts per cell-line (`CELL_LINE_NAME`) different strongly between the two databases.
- Outliers are mostly in the directions of smaller counts. This means, that it is more unlikely that cell-lines have relatively low counts.

In [ ]:
describes = []
for db in ['GDSC1', 'GDSC2']:
    print(f"\nDatabase `{db}`\n{15*'-'}")
    describe = gdsc_cell_line_value_counts[gdsc_cell_line_value_counts.DATASET==db].counts.describe()
    describes.append(describe)
    print(describe)

In [ ]:
print(f"The cell-lines (`{col}`) for")
for i, db in enumerate(['GDSC1', 'GDSC2']):
    print(f""" 
        - {db} have
            - mostly (IQR) between {round(describes[i]['25%'])} and {round(describes[i]['75%'])} observations
            - on average {round(describes[i]['mean'])} observations per cell-line. This corresponds to {100*round(describes[i]['mean'])/gdsc_ic50s_join[gdsc_ic50s_join.DATASET==db].shape[0]:2.2f}% out of all observations ({gdsc_ic50s_join[gdsc_ic50s_join.DATASET==db].shape[0]}) in {db}.
    """)

3. Investigation of the distribution of IC50 values.

In [ ]:
# Compare general IC50 distribution for both GDSC datasets.
print("IC50 Distributions:")
describes = []
for db in ['GDSC1', 'GDSC2']:
    print(f"\nDatabase `{db}`\n{15*'-'}")
    describe = np.exp(gdsc_ic50s_join[gdsc_ic50s_join.DATASET==db].LN_IC50).describe()
    describes.append(describe)
    print(describe)

In [ ]:
print(f"The IC50 values for")
for i, db in enumerate(['GDSC1', 'GDSC2']):
    print(f""" 
        - {db} have
            - mostly (IQR) a value between {round(describes[i]['25%'])} and {round(describes[i]['75%'])}
            - on average a value of {round(describes[i]['mean'])}.
    """)

_Note_: 
- If the IC50 value is high, this indicates that the cell line developed a resistent to the agent (drug).

In [ ]:
# Distribution of the IC50 values.

figure, axs = plt.subplots(2, 2, figsize=(15, 10))
figure.suptitle(f"Distributions of the ln(IC50) and IC50 values \n for GDSC1 & GDSC2")

sns.boxplot(data=gdsc_ic50s_join, y='DATASET', x='LN_IC50', linewidth=2, ax=axs[0, 0]);
axs[0, 0].set_xlabel(f"ln(IC50)");
sns.boxplot(y=gdsc_ic50s_join.DATASET, x=np.exp(gdsc_ic50s_join.LN_IC50), linewidth=2, ax=axs[0, 1]);
axs[0, 1].set_xlabel(f"IC50");
sns.kdeplot(data=gdsc_ic50s_join, hue='DATASET', x='LN_IC50', linewidth=2, ax=axs[1, 0]);
axs[1, 0].set_xlabel(f"ln(IC50)");
sns.kdeplot(hue=gdsc_ic50s_join.DATASET, x=np.exp(gdsc_ic50s_join.LN_IC50), linewidth=2, ax=axs[1, 1]);
axs[1, 1].set_xlabel(f"IC50");

- The distribution of the $ln(IC50)$ values is very similar between the two databases GDSC1 and GDSC2.
- There are a lot of outliers existent. Thus many cell-lines developed resistence against a specific drug. However, for now it is unclear if this conclusion can also be made for very few cell-lines but a lot of drugs or for many cell-lines but then only for a few drugs.

### Raw Data

In [ ]:
GDSC1_RAW_FILE = 'GDSC1_public_raw_data_25Feb20.csv'
GDSC2_RAW_FILE = 'GDSC2_public_raw_data_25Feb20.csv'

In [ ]:
# Read the raw files.

# GDSC1
start = time.time()
gdsc1_raw = pd.read_csv(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC1_RAW_FILE}', header=0)
print(f"File `{GDSC1_RAW_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc1_raw.shape}")

# GDSC2
start = time.time()
gdsc2_raw = pd.read_csv(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC2_RAW_FILE}', header=0)
print(f"File `{GDSC2_RAW_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc2_raw.shape}")

In [ ]:
gdsc1_raw.head(5)

In [ ]:
gdsc2_raw.head(5)

- GDSC1-raw contains $5,837,703$ rows
- GDSC2-raw contains $6,646,430$ rows

Each dataset contains the same $18$ columns. They hold the following informations (which were taken from [here](ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC_Raw_Data_Description.pdf)):

- A description of the columns can be found in `knowledge_base/table_contents.md` under `Drug Screening - Raw data`.

The table contains information about the concentration and intensity of a drug (`DRUG_ID`) in a specific cell line (`CELL_LINE_NAME`). Specifically it contains infromation about experiments which lead to this information.

In [ ]:
# Join both datasets for analysis purposes.
gdsc_raw_join = pd.concat([gdsc1_raw, gdsc2_raw], ignore_index=True)
assert gdsc_raw_join[gdsc_raw_join.index.duplicated()].shape[0] == 0
assert gdsc_raw_join.shape[0] == gdsc1_raw.shape[0] + gdsc2_raw.shape[0]

In [ ]:
columns = ['RESEARCH_PROJECT', 'CELL_LINE_NAME', 'DRUG_ID']

for col in columns:
  gdsc1_uniq, gdsc2_uniq = np.unique(gdsc1_raw[col]), np.unique(gdsc2_raw[col])

  print(f"""
      Number of unique {col}'s 
          - for GDSC1 is {gdsc1_uniq.size}
          - for GDSC2 is {gdsc2_uniq.size}
          - in total for both is {set(gdsc1_uniq.tolist() + gdsc2_uniq.tolist()).__len__()}
            Thus, {(gdsc1_uniq.size+gdsc2_uniq.size) - set(gdsc1_uniq.tolist() + gdsc2_uniq.tolist()).__len__()} {col}'s are contained in both databases (since the full join is {(gdsc1_uniq.size+gdsc2_uniq.size)}).
  """)

- We can see that for the columns `CELL_LINE_NAME` & `DRUG_ID` the numbers of unique values for each database __are exactly the same as they were for the IC50 files from the `IC50s` section from above__.

To validate that the cell-lines and drug ID's are actually the same we check if there are even any cell-lines and/or drugs which are in the raw files but not in the ic50 files, or vice versa.

In [ ]:
assert not set(np.unique(gdsc1_raw.CELL_LINE_NAME)) - set(np.unique(gdsc1_ic50s.CELL_LINE_NAME))
assert not set(np.unique(gdsc2_raw.CELL_LINE_NAME)) - set(np.unique(gdsc2_ic50s.CELL_LINE_NAME))

Thus, 
- the table in `GDSC1_fitted_dose_response_25Feb20.xlsx` holds exactly the same set of cell-lines (`CELL_LINE_NAME`) & drugs (`DRUG_ID`) as `GDSC1_public_raw_data_25Feb20.csv`.
- the table in `GDSC2_fitted_dose_response_25Feb20.xlsx` holds exactly the same set of cell-lines (`CELL_LINE_NAME`) & drugs (`DRUG_ID`) as `GDSC2_public_raw_data_25Feb20.csv`.

> __Conclusion__: we can inner join the raw table with the ic50 table on either cell-lines or drugs, without loosing any observations.

---
## Join GDSC Data

In this step we join GDSC raw data with the IC50 data.

In [ ]:
# First, we take a look at both - already joined (GDSC1 & GDSC2) - tables. 
# Raw table, from GDSC{1,2}_public_raw_data_25Feb20.csv
gdsc_raw_join.head(3)

In [ ]:
# IC50s table, from GDSC{1,2}_fitted_dose_response_25Feb20.xlsx
gdsc_ic50s_join.head(3)

In [ ]:
# Check for columns which are in both tables. We could join on these columns.
list(set(gdsc_raw_join.columns) - (set(gdsc_raw_join.columns) - set(gdsc_ic50s_join.columns)))

We will join on the tuple (`CELL_LINE_NAME`, `DRUG_ID`). 
- We left join the raw data on the IC50 data to have possibly the concentration of the drug. 

In [ ]:
cols_to_join_on = ['CELL_LINE_NAME', 'DRUG_ID']
gdsc_ic50_raw_join = gdsc_ic50s_join.merge(gdsc_raw_join,
                                           on=cols_to_join_on,
                                           how='left',
                                           suffixes=['_ic50', '_raw'])
print(gdsc_ic50_raw_join.shape)

if gdsc_ic50_raw_join.shape[0] > gdsc_ic50s_join.shape[0]:
    print(f"""There are multiple {cols_to_join_on} entries in the raw GDSC table which match with the IC50 GDSC table.
        number of rows after left join   : {gdsc_ic50_raw_join.shape[0]}
        number of rows in the IC50 table : {gdsc_ic50s_join.shape[0]}
        number of rows in the raw table  : {gdsc_raw_join.shape[0]}
    """)

In [ ]:
gdsc_ic50_raw_join.head(3)

In [ ]:
gdsc_ic50_raw_join.columns

We can exlude the following columns since they probably are not useful as features for building a model.


In [ ]:
cols_to_exclude = [
    # From IC50 table.
    'NLME_RESULT_ID',
    'NLME_CURVE_ID',
    'SANGER_MODEL_ID',
    'TCGA_DESC',
    'PUTATIVE_TARGET',
    'PATHWAY_NAME',
    'COMPANY_ID',
    'MIN_CONC',
    'MAX_CONC',
    # From RAW table.
    'WEBRELEASE',
    'RESEARCH_PROJECT',
    'BARCODE', 
    'SCAN_ID', 
    'DATE_CREATED',
    'SCAN_DATE',
    'COSMIC_ID_raw',
    'DRUGSET_ID',
    'ASSAY',
    'TAG'
]

In [ ]:
gdsc_sparsed_cols = gdsc_ic50_raw_join[list(set(gdsc_ic50_raw_join.columns) - set(cols_to_exclude))]
gdsc_sparsed_cols = gdsc_sparsed_cols.rename(columns={'COSMIC_ID_ic50': 'COSMIC_ID'})
gdsc_sparsed_cols.shape

In [ ]:
gdsc_sparsed_cols.head(10)

- There are clearly many observations per (`CELL_LINE_NAME`, `DRUG_ID`) since there are now different values for different concentrations. 

Let's check in the following how the number of duplicated rows, which only differ in `CONC` and `INTENSITY`, are distributed.

In [ ]:
# Distribution of counts per ('MASTER_CELL_ID', 'DRUG_ID', 'LN_IC50').
group_cols = ['MASTER_CELL_ID', 'DRUG_ID', 'LN_IC50']

figure, axs = plt.subplots(1, 2, figsize=(15, 5))
figure.suptitle(f"Counts of Rows per {group_cols} Group for GDSC1 & GDSC2")

sns.kdeplot(data=gdsc_sparsed_cols.groupby(group_cols).size().reset_index(name='Counts'), 
            x='Counts', linewidth=2, ax=axs[0]);
axs[0].set_title("Counts per group with Outliers");

sns.boxplot(data=gdsc_sparsed_cols.groupby(group_cols).size().reset_index(name='Counts'), 
            x='Counts', showfliers=False, ax=axs[1]);
axs[1].set_title("Counts per group without Outliers");

- Mostly we have between 7 and 12 observations per group.
- This means there are mostly between 7 and 12 different concentrations and/or intensities per cell-line, drug tuple.

In [ ]:
# Since we don't want duplicates with the same LN_IC50 information in our data we will only take the rows with the maximal concentations.
gdsc_without_duplicates = gdsc_sparsed_cols[gdsc_sparsed_cols.groupby(group_cols)['CONC'].transform('max') == gdsc_sparsed_cols['CONC']]
gdsc_without_duplicates = gdsc_without_duplicates[gdsc_without_duplicates.groupby(group_cols)['INTENSITY'].transform('max') == gdsc_without_duplicates['INTENSITY']]
gdsc_without_duplicates.shape

In [ ]:
gdsc_without_duplicates.head(10)

### Further Pre-Processing on GDSC

In [ ]:
# Percent of NaN values per column
100 * gdsc_without_duplicates.isna().sum() / gdsc_without_duplicates.shape[0]

- `SEEDING_DENSITY` has nearly 40% missing values
- `DURATION` has more then 25% missing values
- Both columns can be removed

In [ ]:
gdsc_v2 = gdsc_without_duplicates.loc[:, ~gdsc_without_duplicates.columns.isin(['SEEDING_DENSITY', 'DURATION'])]
gdsc_v2.shape

In [ ]:
gdsc_v2.head(5)

> This dataset `gdsc_v2` can now be used to join features on.

---

# Cell-lines

In [ ]:
!pwd
!ls -lh ../../datasets/gdsc/screening_data | grep '/\|xlsx$' | sort

In this section we will investigate the `Cell_Lines_Details.xlsx` file.

In [ ]:
CELL_LINE_DETAILS_FILE = 'Cell_Lines_Details.xlsx'

In [ ]:
start = time.time()
gdsc_cellline_details = pd.read_excel(f'{PATH_TO_GDSC_SCREENING_DATA}{CELL_LINE_DETAILS_FILE}', header=0)
print(f"File `{CELL_LINE_DETAILS_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc_cellline_details.shape}")
gdsc_cellline_details.head(5)

- The complete table contains information about all screened cell-lines.
- Every row contains information about one cell-line.

---

# Features

In this section we are trying to map feature information to the tables from above. A list of data resources for such features can be found in [here](https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Home.html).

In [ ]:
PATH_TO_GDSC_DATASETS = '../../datasets/gdsc/'

## Gene Expression Data

In [ ]:
!pwd
!ls -lh ../../datasets/gdsc | grep '/\|[xlsx,csv,txt,zip]$' | sort

The file `Cell_line_RMA_proc_basalExp.txt` is extracted from the zip file provided under the _Dataset_ link in [here](https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Home.html) for the entry

| Omic | DataType | Objects | Keywords | Details | Data item | 
| ---- | -------- | ------- | -------- | ------- | --------- | 
| `EXP`| Preprocessed | Cell-lines | RMA normalised expression data for cell-lines | RMA normalised basal expression profiles for all the cell-lines | [Dataset](https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources//Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip) |

In [ ]:
start = time.time()
gdsc_gene_expression = pd.read_csv(PATH_TO_GDSC_DATASETS + 'Cell_line_RMA_proc_basalExp.txt', sep="\t")
print(f"File `Cell_line_RMA_proc_basalExp.txt` took {time.time()-start:.5f} seconds to import. It has shape {gdsc_gene_expression.shape}")
gdsc_gene_expression.head(3)

The table in `Cell_line_RMA_proc_basalExp.txt` contains
- information about the gene expression for specific `COSMIC_ID`'s 

## Mapping

In [ ]:
from external.gdsc_utils.GDSC_utils import get_gdsc_gene_expression

In [ ]:
from typing import Optional, Set
import io 
from pandas._libs.parsers import ParserError

def get_gdsc_gene_expression(
    
    genes: Optional[Set] = None,
    path_cell_annotations: str = "data/GDSC/Cell_Lines_Details.csv",
    path_gene_expression: str = "data/GDSC/Cell_line_RMA_proc_basalExp.txt",
):
    """
    Return the gene expression dataframe(n_cells x n_genes)
    for a set of gene symbols for all cell_lines of the GDSC cell line annotation file.
    If the genes are None, return the data for all genes.
    """

    gene_expression = pd.read_csv(path_gene_expression, sep="\t")

    gene_expression = gene_expression.rename(columns={'GENE_SYMBOLS': 'Sample Name'}, level=0)

    gene_expression = gene_expression.drop(["GENE_title"], axis=1).set_index(
        "Sample Name"
    )
    gene_expression.index = gene_expression.index.astype(str)

    # refactor column names to cosmic id and then map to cell-line name
    ge_columns = [
        x.split("DATA.")[1] for x in list(gene_expression.columns)
    ]  # remove "DATA" prefix
    ge_columns = cosmic_ids_to_cell_line_names(
        ge_columns, path_cell_annotations=path_cell_annotations
    )
    gene_expression.columns = ge_columns.astype(str)

    if genes is None:
        return gene_expression.T
    else:
        # filter out the genes
        number_of_queried_genes = len(genes)
        genes = set(gene_expression.index) & genes
        print(
            f"no data for {number_of_queried_genes - len(genes)} of {number_of_queried_genes} queried genes."
        )
        gene_expression = gene_expression.loc[genes]

    return gene_expression.T

def cosmic_ids_to_cell_line_names(
    cosmic_ids, path_cell_annotations="data/GDSC/Cell_Lines_Details.csv"
):
    """
    transform a list of COSMIC ID's to a series of cell-line-names, indexed by the cosmic ID
    using the cell annotations from https://www.cancerrxgene.org/downloads/bulk_download

    """

    try:
        if path_cell_annotations[-4:] == '.csv': 
            cell_line_data = pd.read_csv(path_cell_annotations, index_col=0)
        elif path_cell_annotations[-5:] == '.xlsx':
            cell_line_data = pd.read_excel(path_cell_annotations)
    except ParserError:
        csv_data = open(path_cell_annotations).read().replace("\r\n", "\n")
        cell_line_data = pd.read_csv(io.StringIO(csv_data), encoding="unicode_escape")

    cosmic_ids_to_cell_line_name_dict = pd.Series(
        cell_line_data["Sample Name"].values,
        index=cell_line_data["COSMIC identifier"].fillna(-1).astype(int).values,
    ).to_dict()

    cell_line_names = []
    unknown_cell_line_names = []
    for cosmic_id in cosmic_ids:
        try:

            cell_line_names.append(cosmic_ids_to_cell_line_name_dict[int(cosmic_id)])

        except (KeyError, ValueError):
            cell_line_names.append("unknown_cosmic_" + str(cosmic_id))
            unknown_cell_line_names.append(cosmic_id)

    if unknown_cell_line_names:
        print(
            "Note: "
            + str(len(unknown_cell_line_names))
            + " Cosmic IDs not found in cell annotation data: "
        )
        print(unknown_cell_line_names)

    # check if cell_line_names are unique
    unique_c = []
    dup_c = []
    for c in cell_line_names:
        if not (c in unique_c):
            unique_c.append(c)
        else:
            dup_c.append(c)
    if dup_c:
        print(
            "Warning: at least two cosmic IDs map to the same cell lines for the cell lines: "
        )
        print(dup_c)

    return pd.Series(cell_line_names, index=cosmic_ids)

In [ ]:
# Return the gene expression dataframe(n_cells x n_genes) for a set of gene symbols for all cell_lines of the GDSC cell line annotation file.
# If the genes are None, return the data for all genes.
gene_expr = get_gdsc_gene_expression(
    path_cell_annotations=f'{PATH_TO_GDSC_SCREENING_DATA}{CELL_LINE_DETAILS_FILE}',
    path_gene_expression=PATH_TO_GDSC_DATASETS + 'Cell_line_RMA_proc_basalExp.txt'
)

In [ ]:
gene_expr.shape

- There are $1,018$  cell-lines (rows in `gene_expr`)
- Per cell-line there are gene expressions of $17,736$ genes (columns in `gene_expr`)

In [ ]:
gene_expr.head(5)

- Each `COSMIC_ID` has a corresponding `SAMPLE Name` in the `Cell_Lines_Details.xlsx` file, which is in the `gdsc_cellline_details` variable.
- Each column in the variable `gene_expr` thus now holds per column the `COSMIC_ID` or `SAMPLE Name` respectively.
- Rememeber: that `COSMIC_ID` <-> `MASTER_CELL_ID`
- Each cell-line (`SAMPLE_NAME`) contains in the row gene expression levels for different genes. 

> __Summary__: The dataset `gene_expr` holds now feature information in the sense of the gene expression level (`TSPAN6`, `TNMD`, ...) per cell-line (`Sample Name`).

In [ ]:
# The `gene_expr` table is a map from the cell-lines to the gene expression. It uses the the following two tables for that:
# - gdsc_cellline_details : from 'Cell_Lines_Details.xlsx'
# - gdsc_gene_expression  : from 'Cell_line_RMA_proc_basalExp.txt'
gdsc_cellline_details.head(5)

In [ ]:
# Here, except the 1st column, all other columns have as headers 'DATA.<COSMIC identifier>' where COSMIS identifier can be mapped to the Cell Line Details table.
gdsc_gene_expression.loc[:, gdsc_gene_expression.columns != 'GENE_title'].head(5)

## Sparsing the Feature Space

Since the `gene_expr` table holds $17,737$ genes (columns) we are trying to sparse down this set of columns by using _LINCS landmark gene symbols_.

- LINCS landmark gene symbols are in file `landmark_genes.csv`

In [ ]:
FILENAME_LANDMARK_GENES = 'landmark_genes.csv' 

In [ ]:
start = time.time()
landmark_genes = pd.read_csv(f'{PATH_TO_GDSC_DATASETS}{FILENAME_LANDMARK_GENES}', sep="\t")
print(f"File `{FILENAME_LANDMARK_GENES}` took {time.time()-start:.5f} seconds to import. It has shape {landmark_genes.shape}")
landmark_genes.head(3)

In [ ]:
gene_expr.head(3)

In [ ]:
# Check how many cell line columns of the gene expressions table are in the landmark gene file.

count, cols_to_keep = 0, []
for c in gene_expr.columns[gene_expr.columns != 'nan']:
    if c in landmark_genes.Symbol.tolist(): 
        count += 1
        cols_to_keep.append(c)
        
print(f"""
    Out of {len(gene_expr.columns[gene_expr.columns != 'nan'])} non-nan columns in the gene expression file (`gene_expr`) {count} columns are respresented  in the landmark_genes.csv file.
    Thus, {100*(1-count/len(gene_expr.columns[gene_expr.columns != 'nan'])):2.2f}% will get removed.
""")


Now we sparse down the columns in `gene_expr` from $17,419$ to all the ones we found in the `landmark_genes.csv` file und the `Symbol` column. This leaves us with $908$ columns. The remaining $94.79$% of all columns in `gene_expr` will get removed. 

In [ ]:
gene_expr_sparse = gene_expr[cols_to_keep]
assert gene_expr_sparse.shape[1] == len(cols_to_keep)
gene_expr_sparse.head(5)

__Summary__:
> `gene_expr_sparse` - Cell lines as index column and genes as columns, where the genes got sparsed down by using landmark gene information.

## Map Gene Expression to GDSC

In this step we are going to map the gene expression information from the file `gene_expr_sparse` to the GDSC tables data. The objective is to have gene expression information as a feature for predicting IC50 values. We will join the two tables on the cell-line. 
- In the GDSC dataset the cell line is decoded in `CELL_LINE_NAME`. 
- In the gene expression dataset the cell line is in the index `Sample Name`.

> __Goal__: Map the gene expression values to the IC50 values.

In [ ]:
gdsc_v2.head(3)

In [ ]:
gene_expr_sparse.head(3)

In [ ]:
col = 'CELL_LINE_NAME'

uniq_gdsc, uniq_gene_expr = np.unique(gdsc_v2[col]), np.unique(gene_expr_sparse.index)
print(f"""
    Number of unique {col}'s 
        in the GDSC db                 : {len(uniq_gdsc)}
        in the gene expression dataset : {len(uniq_gene_expr)}

    Number of cell-lines in the GDSC which are not in the Gene Expression dataset: {len(set(uniq_gdsc) - set(uniq_gene_expr))}. 
    Thus, there will be no gene expression information for these cell-lines.
""")


In [ ]:
gene_expr_sparse.columns.name

In [ ]:
gene_expr_sparse.index

In [ ]:
gene_expr_sparse.loc['CAL-120']

In [ ]:
cols_to_join_on = ['CELL_LINE_NAME']
join_gdsc_geneexpr = gdsc_v2.merge(right       = gene_expr_sparse,
                                   left_on     = cols_to_join_on,
                                   right_index = True,
                                   how         = 'left',
                                   suffixes    = ['_gdsc', '_geneexpr'])
print(join_gdsc_geneexpr.shape)

In [ ]:
join_gdsc_geneexpr.head(10)

In [ ]:
# As an example, here we can see that now multiple equal gene expression values are in the table for the same CELL_LINE_NAME.
join_gdsc_geneexpr[join_gdsc_geneexpr.CELL_LINE_NAME=='MC-CAR'].head(10)

- `join_gdsc_geneexpr` now contains the IC50 values for cell-line, drug combinations and in addition the gene expression values of the specific cell-line to multiple genes. 

In [ ]:
# Save the GDSC table with the gene expression information to a file.
join_gdsc_geneexpr.to_pickle(f'{PATH_TO_SAVE_DATA_TO}joined_gdsc_geneexpr.pkl')

---

# Summary of all Tables

| Dataset Name | Path | Important Columns | Description |
| ------------ | ---- | ----------------- | ----------- |
| `landmark_genes.csv` | `../datasets/gdsc` |  | |
| `GDSC_compounds_inchi_key_with_smiles.csv` |  `../datasets/gdsc` | | |
| `Cell_line_RMA_proc_basalExp.txt` |  `../datasets/gdsc` | - `Gene Symbols` <br> - `DATA.<COSMIC Identifier>` columns | Contains information about the gene expression for specific `COSMIC_ID`'s. |
| `GDSC1_public_raw_data_25Feb20.csv` | `../datasets/gdsc/screening_data` | - `CELL_LINE_NAME` <br> - `COSMIC_ID` <br> - `DRUG_ID` <br> - `CELL_ID` <br> - `MASTER_CELL_ID` <br> - `CONC` | Contains information about the concentration and intensity of a drug (`DRUG_ID`) in a specific cell line (`CELL_LINE_NAME`) and the experiments which lead to this information. |
| `GDSC2_public_raw_data_25Feb20.csv` | `../datasets/gdsc/screening_data` | - `CELL_LINE_NAME` <br> - `COSMIC_ID` <br> - `DRUG_ID` <br> - `CELL_ID` <br> - `MASTER_CELL_ID` <br> - `DRUG_ID` <br> - `CONC` | Contains information about the concentration and intensity of a drug (`DRUG_ID`) in a specific cell line (`CELL_LINE_NAME`) and the experiments which lead to this information. |
| `GDSC1_fitted_dose_response_25Feb20.csv` | `../datasets/gdsc/screening_data` | - `CELL_LINE_NAME` <br> - `COSMIC_ID` <br> - `DRUG_ID` <br> - `LN_IC50` <br> - `AUC` | contain the IC50 values (`LN_IC50`) for specific drugs (`DRUG_NAME`) on cancer cell lines (`CELL_LINE_NAME`). Thus, they hold information about how specific drugs influence the killing of cancer cells. |
| `GDSC2_fitted_dose_response_25Feb20.csv` | `../datasets/gdsc/screening_data` | - `CELL_LINE_NAME` <br> - `COSMIC_ID` <br> - `DRUG_ID` <br> - `LN_IC50` <br> - `AUC` | contain the IC50 values (`LN_IC50`) for specific drugs (`DRUG_NAME`) on cancer cell lines (`CELL_LINE_NAME`). Thus, they hold information about how specific drugs influence the killing of cancer cells. |
| `Cell_line_Details.xlsx` | `../datasets/gdsc/screening_data` | - `Sample Name` <br> - `COSMIC identifier` | The complete table contains information about all screened cell-lines. Every row contains information about one cell-line. |